# LDA

In [ ]:
pip install pandas

In [ ]:
import gensim
from gensim import corpora
from gensim.models import LdaModel
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pandas as pd

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DataAbstrak.csv')

In [ ]:
print(data)

     Unnamed: 0                                            Abstrak
0             0  maka dan minuman khas merupakan ciri dari keka...
1             1  abstrak akhir akhir ini semakin maraknya pencu...
2             2  penelitian ini bertujuan untuk dapat mengetahu...
3             3  perairan soccah merupakan lokasi nelayan menca...
4             4  abstrak rian pandi pranata nim program studi s...
..          ...                                                ...
140         140  analisis bahasa figuratif yang digunakan dalam...
141         141  abstrak tujuan dari penelitian ini adalah untu...
142         142  penelitian ini dilatarbelakangi oleh kesulitan...
143         143  abstrak abstrak penelitian ini mengkaji bentuk...
144         144  abstrak produksi garam krosok di salt house me...

[145 rows x 2 columns]


In [ ]:
# Mengambil Semua Nilai Dalam Kolom Abstrak Dari Data, Disimpan ke dalam bentuk list
documents = data['Abstrak'].tolist()

In [ ]:
# Instal Modul NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english')) # Memberhentikan kata2 berhenti/kata2 umum
lemmatizer = WordNetLemmatizer() # Mengurangi Variasi Kata
tokenized_documents = [] # Membuat List Kosong

# PraPemrosesan Untuk Menyederhanakan Kata
for doc in documents:
    words = word_tokenize(doc.lower())
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]
    tokenized_documents.append(lemmatized_words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
dictionary = corpora.Dictionary(tokenized_documents)  # Membuat kamus/daftar kata2 dan diberi indeks numerik

In [ ]:
corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents] # Mengubah Dokumen ke dalam representasi Vector BoW

In [ ]:
lda_model = LdaModel(corpus, num_topics=3, id2word=dictionary, passes=15) # Melatih model LDA

In [ ]:
for idx, topic in lda_model.print_topics():
    print(f"Topik #{idx + 1}: {topic}")

Topik #1: 0.029*"dan" + 0.025*"yang" + 0.020*"penelitian" + 0.018*"ini" + 0.017*"dengan" + 0.013*"dalam" + 0.011*"di" + 0.011*"dari" + 0.011*"pada" + 0.010*"untuk"
Topik #2: 0.029*"yang" + 0.029*"dan" + 0.016*"dengan" + 0.015*"penelitian" + 0.012*"ini" + 0.011*"dalam" + 0.010*"untuk" + 0.008*"pada" + 0.008*"data" + 0.008*"di"
Topik #3: 0.022*"yang" + 0.017*"dan" + 0.013*"penelitian" + 0.013*"dengan" + 0.012*"dalam" + 0.010*"ini" + 0.008*"risiko" + 0.008*"pada" + 0.008*"kerja" + 0.007*"undang"


In [ ]:
# Membuat DataFrame untuk menampilkan proporsi topik dalam dokumen
document_topic_df = pd.DataFrame()

for doc in corpus:
    topic_distribution = lda_model.get_document_topics(doc, minimum_probability=0)
    doc_topic_props = {f"Topik {topic_id + 1}": prop for topic_id, prop in topic_distribution}
    doc_topic_props = {} #mengubah tampilan agar topik di probalility hilang dan ada pada tabel diatasnya
    for topic_id, prob in topic_distribution:
        key = f"Topik {topic_id + 1}"
        doc_topic_props[key] = prob
    document_topic_df = pd.concat([document_topic_df, pd.Series(doc_topic_props)], ignore_index=True, axis=1)

document_topic_df = document_topic_df.transpose()  # Transpose agar topik menjadi kolom

# Menampilkan tabel proporsi topik dalam dokumen
print("Tabel Proporsi Topik dalam Dokumen:")
print(document_topic_df)

Tabel Proporsi Topik dalam Dokumen:
      Topik 1   Topik 2   Topik 3
0    0.996069  0.001993  0.001938
1    0.001440  0.001403  0.997157
2    0.459788  0.537571  0.002642
3    0.002254  0.995677  0.002069
4    0.995903  0.002020  0.002078
..        ...       ...       ...
140  0.002182  0.995825  0.001993
141  0.993431  0.003262  0.003307
142  0.001788  0.120710  0.877502
143  0.002368  0.002339  0.995293
144  0.995460  0.002299  0.002241

[145 rows x 3 columns]


In [ ]:
# Membuat DataFrame untuk menampilkan proporsi kata dalam topik
topic_word_df = pd.DataFrame()

for topic_id in range(lda_model.num_topics):
    topic_words = lda_model.show_topic(topic_id, topn=10)  # mengambil 10 kata kunci teratas
    words_list = [word for word, _ in topic_words]
    words_list = []
    for word, bbt in topic_words:
        words_list.append(word)
    topic_word_df[f"Topik {topic_id + 1}"] = words_list

# Menampilkan tabel proporsi kata dalam topik
print("\nTabel Proporsi Kata dalam Topik:")
print(topic_word_df)


Tabel Proporsi Kata dalam Topik:
      Topik 1     Topik 2     Topik 3
0         dan        yang        yang
1        yang         dan         dan
2  penelitian      dengan  penelitian
3         ini  penelitian      dengan
4      dengan         ini       dalam
5       dalam       dalam         ini
6          di       untuk      risiko
7        dari        pada        pada
8        pada        data       kerja
9       untuk          di      undang
